In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from collections import defaultdict, namedtuple
import os 
import sys
import glob
import scipy.misc as spm
import json
import pprint
import pandas as pd
from PIL import Image
from sklearn.feature_extraction.image import extract_patches_2d
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import CSVLogger, ModelCheckpoint, ReduceLROnPlateau

In [ ]:
train_data = pd.read_csv('../data/validation/train.csv')

In [ ]:
example = train_data.iloc[0]
print example.filename
print example.label

filename = '../data/train/{}/{}'.format(example.label, example.filename)
image = spm.imread(filename)
plt.imshow(image)
plt.show()

In [ ]:
data = open('../data/bounding_boxes/{}_labels.json'.format(example.label.lower()))
bb = json.load(data)
patch_width = 128
patch_height = 128
nb_epoch = 10
print image.shape

In [ ]:
DATA_FOLDER = '../Data'
bboxes = defaultdict(list)
files = glob.glob(DATA_FOLDER + '/bounding_boxes/*')
for ff in files:
    with open(ff) as f:
        data = json.load(f)
        for image in data:
            for annot in image['annotations']:
                bboxes[DATA_FOLDER + '/train/' + image['filename']].append((max(0, annot['x']), max(0, annot['y']), annot['width'], annot['height']))

In [ ]:
for img in bb:
    for annot in img['annotations']:
        sum_width +=  annot['width']
        sum_height +=  annot['height']
        bboxes[img['filename']].append((annot['x'], annot['y'], annot['width'], annot['height']))

        
        

In [ ]:
def get_patches(bbox, amount= 1):
    return extract_patches_2d(bbox, patch_size=(patch_height,patch_width), max_patches=amount, random_state=None)


In [ ]:
def DoBoxesIntersect(a,b):
  return (abs(a.x - b.x) * 2 < (a.width + b.width)) and (abs(a.y - b.y) * 2 < (a.height + b.height))

In [ ]:
count = 0
for k,v in bboxes.iteritems():
    print v
    count += 1
    if count > 10:
        break

In [ ]:
X_train = []
y_train = []
X_test = []
y_test = []

for k,v in bboxes.iteritems():
    image = spm.imread(k)
    
    box = namedtuple('box', ['x','y','width','height'])    
    fish_boxes = []
    
    
    for b in v:
        x = int(round(b[0]))
        y = int(round(b[1]))
        width = int(round(b[2]))
        height = int(round(b[3]))
        fish_boxes.append(box(x,y,width,height))

    x = fish_boxes[0].x
    y = fish_boxes[0].y
    width = fish_boxes[0].width
    height = fish_boxes[0].height
    bbox_fish_image = image[y:y+height,x:x+width,:]    
        
    count = 0
    patches_rnd = []
    while count < 5:
        img_y = np.random.randint(image.shape[0]-patch_height)
        img_x = np.random.randint(image.shape[1]-patch_width)
        box_rnd = box(img_x, img_y, patch_width,patch_height)
        intersects = False
        for fish_box in fish_boxes:
            if DoBoxesIntersect(fish_box, box_rnd):
                intersects = True
                break
        if not intersects:
            patches_rnd.append(image[img_y:img_y+patch_height,img_x:img_x+patch_width,:])
            count += 1
            
    
    if width < patch_width or height < patch_height:
        continue
    
    patches_fish = get_patches(bbox_fish_image,amount=5)
    count = 1
    for p1,p2 in zip(patches_fish,patches_rnd):
        p1 = spm.imresize(p1,size=(64,64)).transpose(2,0,1)
        p2 = spm.imresize(p2,size=(64,64)).transpose(2,0,1)
        if count < len(patches_fish):
            X_train.append(p1)
            y_train.append(1)
            X_train.append(p2)
            y_train.append(0)
        else:
            X_test.append(p1)
            y_test.append(1)
            X_test.append(p2)
            y_test.append(0)
        count +=1
            
            
X_train = np.array(X_train).astype('float32') /255
y_train = np.array(y_train)
X_test = np.array(X_test).astype('float32') /255
y_test = np.array(y_test)


print X_train.shape
print y_train.shape
print X_test.shape
print y_test.shape


In [ ]:
# Example how to extract the good labels
# Only using the first index, should become the largest??
height, width, co_x, co_y = None, None, None, None
with open('../data/bounding_boxes/{}_labels.json'.format(example.label.lower())) as data_file:
    data = json.load(data_file)
    for i in range(len(data)):
        path = '../data/train/{}/{}'.format(example.label, example.filename)
        if data[i]['filename'] == path:
            height = data[i]['annotations'][0]['height']
            width = data[i]['annotations'][0]['width']
            co_x = data[i]['annotations'][0]['x']
            co_y = data[i]['annotations'][0]['y']

fig, ax = plt.subplots(1)
ax.imshow(image)
plt.xlim(co_x, co_x+width)
plt.ylim(co_y, co_y+height)
plt.show()

In [ ]:
for img, i in zip(X_train,y_train):
    if i == 1:
        plt.imshow(img.transpose(1,2,0))
        plt.show()

In [ ]:
def keras_model_conv2():
  from keras.models import Sequential
  from keras.layers import Dense, Dropout, Activation,Flatten
  from keras.layers import Convolution2D, MaxPooling2D
  from keras.optimizers import SGD
  model = Sequential()
  model.add(Convolution2D(16, 5, 5, 
                          border_mode='valid', 
                          input_shape=(3, 64, 64)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Convolution2D(32, 3, 3))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Convolution2D(64, 3, 3))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2,2))) 
  print model.layers[-1].output_shape  
  model.add(Convolution2D(512, 6,6))
  model.add(Activation('relu'))
  model.add(Convolution2D(256, 1, 1))
  model.add(Activation('relu'))
  model.add(Convolution2D(1, 1, 1))
  model.add(Flatten())
  model.add(Activation('sigmoid'))

    
  model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
  
  return model

In [ ]:
model_path = r'C:\Users\Thomas\Documents\MLIP-BigAITuna\model\bounding_boxes.h5'
model = keras_model_conv2()
if os.path.exists(model_path):
    model.load_weights(model_path)


In [ ]:
datagen = ImageDataGenerator(horizontal_flip=True,
                             vertical_flip=True,
                             rotation_range=10,
                             zoom_range=[0.8, 1.2],
                             width_shift_range=0.1,
                             height_shift_range=0.1)
train_gen = datagen.flow(X_train, 
                         y_train)
val_gen = ImageDataGenerator().flow(X_test, y_test)
    

In [ ]:
csv_logger = CSVLogger('run4_adam.csv')
lr_plateau = ReduceLROnPlateau(monitor='val_loss', patience=2, verbose=1, factor=0.9)
checkpoint = ModelCheckpoint(filepath='C:/Users/Thomas/Documents/MLIP-BigAITuna/logging/model.{epoch:02d}-{loss}.hdf5', verbose=1, save_best_only=True)

model.fit_generator(train_gen, samples_per_epoch=len(X_train),
                    nb_epoch=30, verbose=2,
                    callbacks=[csv_logger, lr_plateau, checkpoint],
                    validation_data=val_gen, nb_val_samples=len(X_test))

model.save_weights(model_path)

In [ ]:
def keras_model_conv_test():
  from keras.models import Sequential
  from keras.layers import Dense, Dropout, Activation,Flatten
  from keras.layers import Convolution2D, MaxPooling2D
  from keras.optimizers import SGD
  model = Sequential()
  model.add(Convolution2D(16, 5, 5, 
                          border_mode='valid', 
                          input_shape=(3, None, None)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Convolution2D(32, 3, 3))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Convolution2D(64, 3, 3))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2,2))) 
  print model.layers[-1].output_shape  
  model.add(Convolution2D(512, 6,6))
  model.add(Activation('relu'))
  model.add(Convolution2D(256, 1, 1))
  model.add(Activation('relu'))
  model.add(Convolution2D(1, 1, 1))
  model.add(Activation('sigmoid'))

    
  model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
  
  return model

In [ ]:
model_test = keras_model_conv_test()
model_test.load_weights(model_path)

In [ ]:
# example = train_data.iloc[200]
# filename = '../data/train/{}/{}'.format(example.label, example.filename)
# filename = '../data/train/{}/{}'.format("Alb", "img_01583.jpg")
# image = spm.imread(filename)


image_height = image.shape[0]/2
image_width = image.shape[1]/2
img_resize = spm.imresize(image,size=(image_height,image_width)).transpose(2,0,1).astype('float32')/255
img_resize = img_resize.reshape(1,3,image_height,image_width)

In [ ]:
score = model_test.predict_proba(img_resize)

In [ ]:
heatmap = score[0].transpose(1,2,0).squeeze() >0.99
print heatmap.shape

In [ ]:
plt.imshow(heatmap)
plt.show()
plt.imshow(image)
plt.show()